## 球種予測部門 Tuning
- all_merge_15.f

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import lightgbm as lgb
import optuna.integration.lightgbm as lgb
from sklearn import datasets
from sklearn.model_selection import train_test_split
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

In [2]:
ALL_MERGE = 'intermediate/all_merge_16.f'

In [3]:
all_pitch = pd.read_feather(ALL_MERGE)
print(all_pitch.shape)
all_pitch.head()

(778767, 153)


,No,ball,pitch_area,No_in_game,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,pitch_LR,bat_LR,bc_straight,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,bc_area0,bc_area1,bc_area2,bc_area3,bc_area4,bc_area5,bc_area6,bc_area7,bc_area8,bc_area9,bc_area10,bc_area11,bc_area12,first,second,third,base_cnt,top_bot,role,pos_pit,date_from_opening,...,weight_bat,draft_order_bat,salary_bat,foreigner_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,salary_year_bat,salary_x_year_bat,bmi_bat,batter_cnt,bat_game_cnt,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,bmi_dif_b-c,pit_bat_L_L,pit_bat_L_R,pit_bat_R_L,pit_bat_R_R
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.0,1.0,0.464734,0.110359,0.151086,0.097383,0.067492,0.043210,0.012933,0.052803,0.064837,0.095199,0.082009,0.040512,0.063252,0.053916,0.030534,0.046508,0.038328,0.163762,0.057728,0.130872,0.132543,0,0,0,0,1,1,0,0,...,74,5.0,15000,0,1,1,0,32,8.0,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725,0,0,1,0
1,1,0.0,8.0,2,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.0,1.0,0.442351,0.093302,0.109348,0.078518,0.141711,0.065447,0.017669,0.051654,0.044893,0.071306,0.068422,0.032002,0.043180,0.036149,0.029388,0.042910,0.033625,0.152078,0.078157,0.203642,0.164248,0,0,0,0,1,1,0,0,...,74,5.0,15000,0,1,1,0,32,8.0,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725,0,0,1,0
2,2,0.0,5.0,3,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0.0,1.0,0.504212,0.069963,0.065568,0.046520,0.211722,0.059158,0.019231,0.023626,0.027656,0.045421,0.045238,0.025275,0.023810,0.027106,0.023077,0.032784,0.022711,0.202930,0.081319,0.285165,0.157509,0,0,0,0,1,1,0,0,...,74,5.0,15000,0,1,1,0,32,8.0,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725,0,0,1,0
3,3,0.0,12.0,4,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0.0,0.0,0.475961,0.101792,0.226048,0.066617,0.042131,0.018345,0.003343,0.065762,0.034319,0.043647,0.027090,0.054336,0.065374,0.045046,0.062653,0.097944,0.085507,0.098410,0.098527,0.055307,0.231840,0,0,0,0,1,1,0,0,...,74,1.0,7040,0,0,0,1,40,22.0,320.0,154880.0,22.839506,268.0,81.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681,0,0,0,1
4,4,0.0,8.0,5,5,1,2,2,1,2,5,5,2,1,0,0,1,0,1,0.0,0.0,0.463496,0.099763,0.194714,0.067542,0.091198,0.023901,0.004486,0.054898,0.024390,0.034832,0.025124,0.039481,0.047557,0.033526,0.049596,0.080512,0.069337,0.085243,0.121054,0.076923,0.312424,0,0,0,0,1,1,0,0,...,74,1.0,7040,0,0,0,1,40,22.0,320.0,154880.0,22.839506,268.0,81.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681,0,0,0,1


### train
- 行数: 257117

In [4]:
train = all_pitch.dropna(subset=['course'])
train.shape

(257117, 153)

### test
- 行数: 521650

In [5]:
test = all_pitch[all_pitch['course'].isnull()]
test.shape

(521650, 153)

In [6]:
del all_pitch
gc.collect()

40

In [7]:
train_d = train.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

In [8]:
test_d = test.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

## Dataset作成

In [9]:
X_train, X_test, y_train, y_test = train_test_split(train_d, train['ball'])
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

### LGBM学習
- multiclass : クラス数=8

In [10]:
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'gbdt',
        'metric' : 'multi_logloss',
        'num_class' : 8,
    }

### Tuningしながら学習

In [11]:
%%time
best_params, tuning_history = dict(), list()
lgb_model = lgb.train(lgb_param, lgb_train,
                      valid_sets=lgb_eval,
                      verbose_eval=0,
                      best_params=best_params,
                      tuning_history=tuning_history)
print('Best Params:', best_params)
#print('Tuning history:', tuning_history)

/home/yasunobu_umehara/anaconda3/lib/python3.7/site-packages/optuna/_experimental.py:90: ExperimentalWarning: train is experimental (supported from v0.18.0). The interface can change in the future.
  ExperimentalWarning,
tune_num_leaves, val_score: 1.106968: 100%|##########| 20/20 [49:38<00:00, 148.93s/it]
tune_bagging_fraction_and_bagging_freq, val_score: 1.106968:  10%|#         | 1/10 [01:27<13:09, 87.77s/it][I 2020-05-09 07:10:17,021] Finished trial#0 with value: 1.1128341968131559 with parameters: {'bagging_fraction': 0.5338630238823251, 'bagging_freq': 5}. Best is trial#0 with value: 1.1128341968131559.
tune_bagging_fraction_and_bagging_freq, val_score: 1.106968:  20%|##        | 2/10 [03:03<12:01, 90.23s/it][I 2020-05-09 07:11:53,010] Finished trial#1 with value: 1.108900691735555 with parameters: {'bagging_fraction': 0.7686468201138247, 'bagging_freq': 7}. Best is trial#1 with value: 1.108900691735555.
tune_bagging_fraction_and_bagging_freq, val_score: 1.106968:  30%|###       

tune_lambda_l1_and_lambda_l2, val_score: 1.100379:  85%|########5 | 17/20 [30:28<05:19, 106.54s/it][I 2020-05-09 08:00:19,270] Finished trial#16 with value: 1.1069776273088057 with parameters: {'lambda_l1': 0.010111503155675926, 'lambda_l2': 0.06472330651951665}. Best is trial#11 with value: 1.1003794021526507.
tune_lambda_l1_and_lambda_l2, val_score: 1.100379:  90%|######### | 18/20 [32:10<03:30, 105.26s/it][I 2020-05-09 08:02:01,552] Finished trial#17 with value: 1.1064758292552623 with parameters: {'lambda_l1': 3.777771081115253e-06, 'lambda_l2': 6.037340011805553e-05}. Best is trial#11 with value: 1.1003794021526507.
tune_lambda_l1_and_lambda_l2, val_score: 1.100379:  95%|#########5| 19/20 [33:56<01:45, 105.34s/it][I 2020-05-09 08:03:47,062] Finished trial#18 with value: 1.105239821581344 with parameters: {'lambda_l1': 0.8518917525725237, 'lambda_l2': 2.162950522869242e-07}. Best is trial#11 with value: 1.1003794021526507.
tune_lambda_l1_and_lambda_l2, val_score: 1.100347: 100%|###

Best Params: {'lambda_l1': 9.466286566446003, 'lambda_l2': 0.15842866429240474, 'num_leaves': 21, 'feature_fraction': 0.41600000000000004, 'bagging_fraction': 0.8473186677812694, 'bagging_freq': 7, 'min_child_samples': 100}
CPU times: user 17h 49min 59s, sys: 3min 43s, total: 17h 53min 42s
Wall time: 2h 14min 40s


### Feature Importance

In [12]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(30)

,feat_name,feat_imp
18,bc_straight,1761
137,salary_year_dif_p-c,1801
99,height_bat,1830
100,weight_bat,1841
89,shoot,1880
6,player_cnt_in_game,1920
128,salary_dif_p-b,1938
132,salary_x_year_dif_p-b,1960
109,salary_year_bat,1998
53,point_diff,2025


In [13]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 6min 37s, sys: 2.39 s, total: 6min 40s
Wall time: 53.5 s


## 結果出力

In [14]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 9)


,index,0,1,2,3,4,5,6,7
0,0,0.534553,0.055510,0.138947,0.197898,0.004946,0.000030,0.000018,0.068099
1,1,0.486996,0.086362,0.145003,0.220606,0.007844,0.000050,0.000016,0.053122
2,2,0.319968,0.101208,0.177755,0.147301,0.130786,0.000162,0.000049,0.122771
3,3,0.440024,0.136005,0.178839,0.161915,0.027849,0.000093,0.000038,0.055238
4,4,0.216411,0.085615,0.406753,0.161583,0.020000,0.000089,0.000035,0.109513


### 2017の平均
- ストレート           0.468254
- カーブ             0.074725
- スライダー           0.185806
- シュート            0.070633
- フォーク            0.083013
- チェンジアップ         0.054217
- シンカー            0.011361
- カットボール          0.051992

In [15]:
submit.mean()

index    260824.500000
0             0.520117
1             0.072431
2             0.186197
3             0.052911
4             0.068321
5             0.046201
6             0.007107
7             0.046715
dtype: float64

In [16]:
#submit.to_csv('submit/tuning.csv', header=False, index=False)

### Tuning結果(15)
`
Best Params: {
    'lambda_l1': 9.466286566446003, 
    'lambda_l2': 0.15842866429240474, 
    'num_leaves': 21, 
    'feature_fraction': 0.41600000000000004, 
    'bagging_fraction': 0.8473186677812694, 
    'bagging_freq': 7, 
    'min_child_samples': 100
}
Wall time: 2h 14min 40s
`

### Tuning結果(10)
`
Best Params: {
    'lambda_l1': 2.972300673782286, 
    'lambda_l2': 0.0002499813121238942, 
    'num_leaves': 15, 
    'feature_fraction': 0.4, 
    'bagging_fraction': 1.0, 
    'bagging_freq': 0, 
    'min_child_samples': 100
}
Wall time: 1h 54min 11s
`

### Tuning結果(5)
`
Best Params: {
    'lambda_l1': 6.9923570049658075, 
    'lambda_l2': 0.002378623984798833, 
    'num_leaves': 18, 
    'feature_fraction': 0.45199999999999996, 
    'bagging_fraction': 0.9799724836460725, 
    'bagging_freq': 4, 
    'min_child_samples': 20
}
`